In [1]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 60.5 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.4/770.4 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.4 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 19.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
#!pip install -q -U transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q --upgrade gradio
#!pip install -q -U sentencepiece


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
!git config --global credential.helper store

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer
from threading import Thread

base_model = "TokenBender/codeCherryPy_7B_llama2"
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(base_model, torch_dtype=torch.float16)
# model.config.use_cache is default to `False`. For inference: `model.config.use_cache = True`

model.config.use_cache = True
model = model.to('cuda:0')

class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = [29, 0]
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

def predict(message, history): 

    history_transformer_format = history + [[message, '''[INST]<<SYS>>
 You are a helpful coding assistant, provide code based on the given query in context.
<</SYS>>[/INST]''']]
    stop = StopOnTokens()

    messages = "".join(["".join(["\n<human>:"+item[0], "\n<bot>:"+item[1]])  #curr_system_message + 
                for item in history_transformer_format])
    
    model_inputs = tokenizer([messages], return_tensors="pt").to("cuda")
    streamer = TextIteratorStreamer(tokenizer, timeout=10., skip_prompt=True, skip_special_tokens=True)
    generate_kwargs = dict(
        model_inputs,
        streamer=streamer,
        max_new_tokens=2048,
        do_sample=True,
        top_p=0.95,
        top_k=10,
        temperature=1.0,
        num_beams=1,
        stopping_criteria=StoppingCriteriaList([stop])
        )
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()

    partial_message  = ""
    for new_token in streamer:
        if new_token != '<':
            partial_message += new_token
            yield partial_message 
            

gr.ChatInterface(predict).queue().launch(share=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://75ccea6ea0d348d21c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [8]:
!pip install -q -U numba

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
from numba import cuda
device = cuda.get_current_device()
device.reset()

In [ ]:
import torch
#from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer
from transformers import LlamaTokenizer, LlamaForCausalLM, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer

base_model = "SLAM-group/NewHope"
tokenizer = LlamaTokenizer.from_pretrained(base_model)
model = LlamaForCausalLM.from_pretrained(base_model, torch_dtype=torch.float16, device_map="cuda")
# model.config.use_cache is default to `False`. For inference: `model.config.use_cache = True`

model.config.use_cache = True

# Suppose required tokenizer and model have already been loaded

In [12]:
instruction = "Write a Python program to find the kth smallest number in a given list. Also, make sure that you do it without using any implicit sort function"
prompt = f"<s> ### Instruction:\n{instruction}\n\n### Response:\n"
inputs = tokenizer(prompt, add_special_tokens=False, return_tensors="pt").to("cuda")
output = model.generate(**inputs, do_sample=True, top_p=0.9, max_new_tokens=2048)[0]
decoded_output = tokenizer.decode(output, skip_special_tokens=True).split("### Response:\n")[-1].strip()
print(decoded_output)

Here is a Python program that can solve the problem. This program uses the Quickselect algorithm which is a variant of QuickSort, but it only recurses one level deep so it is faster for finding the kth smallest number in a list.

```python
def quick_select(l, k):
    if len(l) == 1:
        return l[0]

    pivot = l[len(l) // 2]

    lows = [el for el in l if el < pivot]
    highs = [el for el in l if el > pivot]
    pivots = [el for el in l if el == pivot]

    if k < len(lows):
        return quick_select(lows, k)
    elif k < len(lows) + len(pivots):
        return pivots[0]
    else:
        return quick_select(highs, k - len(lows) - len(pivots))

# Testing the function
numbers = [1, 4, 3, 5, 2, 6, 7, 8]
k = 2
print("The kth smallest number is: ", quick_select(numbers, k))
```
This program works by first checking if the list is only contains one element, in which case that element is the kth smallest number. 

If there are more than one element, it picks a pivot element (the middl

In [11]:
instruction = "Write a Python function that takes a list of positive integers and a target integer. It should find two integers in the list such that their sum is closest to the target. If there are multiple pairs, the function may return any of them. Do it in O(nlogn) time complexity."
prompt = f"<s> ### Instruction:\n{instruction}\n\n### Response:\n"
inputs = tokenizer(prompt, add_special_tokens=False, return_tensors="pt").to("cuda")
output = model.generate(**inputs, do_sample=True, top_p=0.9, max_new_tokens=2048)[0]
decoded_output = tokenizer.decode(output, skip_special_tokens=True).split("### Response:\n")[-1].strip()
print(decoded_output)

This problem seems to be a variant of a classic programming challenge: finding the nearest integer to a target value. We will use the "bin_search" technique to find the nearest values, which has O(logn) time complexity. 

The bin_search works by repeatedly dividing the search interval in half. It is a form of divide and conquer strategy which makes the time complexity of this algorithm better than linear search.

Here is the Python function that does that:

```python
def closest_pair(nums, target):
    def search(lo, hi, x):
        if lo > hi:
            return False
        mid = lo + (hi - lo) // 2
        if x < nums[mid]:
            return search(lo, mid - 1, x)
        elif x > nums[mid]:
            return search(mid + 1, hi, x)
        else:
            return True
    def closest_sum(lo, hi):
        if hi < lo:
            return float('-inf')
        mid = lo + (hi - lo) // 2
        val = nums[mid] + nums[hi - 1]
        if val >= target:
            return closest_sum(lo

In [9]:
instruction = "Write a program in Python to find all possible unique subsets of a set of distinct integers in most optimal time complexity. Mention the time and space comp"
prompt = f"<s> ### Instruction:\n{instruction}\n\n### Response:\n"
inputs = tokenizer(prompt, add_special_tokens=False, return_tensors="pt").to("cuda")
output = model.generate(**inputs, do_sample=True, top_p=0.9, max_new_tokens=2048)[0]
decoded_output = tokenizer.decode(output, skip_special_tokens=True).split("### Response:\n")[-1].strip()
print(decoded_output)

You can use the power set concept to generate all possible unique subsets. Here's a Python program for the problem:

```python
from itertools import chain, combinations

def power_set(arr):
    # Generate all subsets
    return list(chain.from_iterable(combinations(arr, i) for i in range(0, len(arr)+1)))

# Test
arr = [1, 2, 3]
print(power_set(arr))
```

This program uses the itertools library to generate all possible unique subsets. The `power_set` function takes a list of integers as input and returns a list of tuples where each tuple is a unique subset of the input list.

In terms of time complexity, the `combinations` function from the itertools module uses a dynamic algorithm (stored in _combinations_count class method), which provides an efficient approach to compute combinations. The time complexity for generating all possible subsets is O(n+k) where n is the number of elements in the input list and k is the number of subsets. This is because for each element in the input list, 

In [2]:
from transformers import AutoTokenizer
import transformers
import torch

model = "TokenBender/llama2-7b-chat-hf-codeCherryPop-qLoRA-merged"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at TokenBender/llama2-7b-chat-hf-codeCherryPop-qLoRA-merged and are newly initialized: ['model.layers.20.self_attn.rotary_emb.inv_freq', 'model.layers.10.self_attn.rotary_emb.inv_freq', 'model.layers.6.self_attn.rotary_emb.inv_freq', 'model.layers.17.self_attn.rotary_emb.inv_freq', 'model.layers.26.self_attn.rotary_emb.inv_freq', 'model.layers.12.self_attn.rotary_emb.inv_freq', 'model.layers.0.self_attn.rotary_emb.inv_freq', 'model.layers.18.self_attn.rotary_emb.inv_freq', 'model.layers.27.self_attn.rotary_emb.inv_freq', 'model.layers.24.self_attn.rotary_emb.inv_freq', 'model.layers.8.self_attn.rotary_emb.inv_freq', 'model.layers.25.self_attn.rotary_emb.inv_freq', 'model.layers.28.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rotary_emb.inv_freq', 'model.layers.4.self_attn.rotary_emb.inv_freq', 'model.layers.14.self_attn.rotary_emb.inv_freq', 'model.layers.13.self_attn.rotary_emb.inv_freq', 'mod

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [4]:
conversation_text = '''[INST]<<SYS>>
 You are a helpful coding assistant that provides code based on the given query in context.
<</SYS>>
Write a python program to perform celsius to fahrenheir conversion.[/INST]'''

sequences = pipeline(
    conversation_text,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=2048,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Result: [INST]<<SYS>>
 You are a helpful coding assistant that provides code based on the given query in context.
<</SYS>>
Write a python program to perform celsius to fahrenheir conversion.[/INST]  Sure, I'd be happy to help! Here is a Python program that performs Celsius to Fahrenheit conversion:
```
def celsius_to_fahrenheit(celsius):
    fahrenheit = (celsius * 9/5) + 32
    return fahrenheit

# Test the function
print(celsius_to_fahrenheit(0))  # prints 32
print(celsius_to_fahrenheit(20))  # prints 68
print(celsius_to_fahrenheit(30))  # prints 86
```
This program defines a function called `celsius_to_fahrenheit` that takes a single argument `celsius`, which is the temperature in Celsius that you want to convert. The function calculates the Fahrenheit equivalent of the Celsius temperature by multiplying the Celsius value by 9/5 and then adding 32. Finally, it returns the calculated Fahrenheit value.
In the program, we also include some test cases to demonstrate how to use the funct

In [5]:
conversation_text = '''[INST]<<SYS>>
 You are a helpful coding assistant that provides code based on the given query in context.
<</SYS>>
Write a C program to perform celsius to fahrenheir conversion.[/INST]'''

sequences = pipeline(
    conversation_text,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=2048,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: [INST]<<SYS>>
 You are a helpful coding assistant that provides code based on the given query in context.
<</SYS>>
Write a C program to perform celsius to fahrenheir conversion.[/INST]  Sure, here is a C program that performs the conversion from Celsius to Fahrenheit:
```
#include <stdio.h>

int main() {
    double celsius = 20; // replace with your desired Celsius value
    double fahrenheit = CelsiusToFahrenheit(celsius);
    printf("The temperature in Fahrenheit is: %f\n", fahrenheit);
    return 0;
}

double CelsiusToFahrenheit(double celsius) {
    return (celsius * 9/5) + 32;
}
```
This program uses the formula `Celsius = (Fahrenheit - 32) * 5/9` to convert from Celsius to Fahrenheit. The formula is implemented in the `CelsiusToFahrenheit()` function, which takes in a double value representing the temperature in Celsius and returns the corresponding temperature in Fahrenheit.
In the `main()` function, the program prompts the user to enter a temperature in Celsius, and the

In [6]:
conversation_text = '''[INST]<<SYS>>
 You are a helpful coding assistant that provides code based on the given query in context.
<</SYS>>
Write a C program to read a given file and count the occurence of 'the' word in it, use secure coding practices.[/INST]'''

sequences = pipeline(
    conversation_text,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=2048,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: [INST]<<SYS>>
 You are a helpful coding assistant that provides code based on the given query in context.
<</SYS>>
Write a C program to read a given file and count the occurence of 'the' word in it, use secure coding practices.[/INST]  Great! I'd be happy to help you with that. Here's a C program that reads a given file and counts the number of occurrences of the word "the":
### Example File Contents
Here's an example of what the file might contain:
```
Hello there! The sun is shining brightly today.
The cat is sleeping on the couch.
The dog is barking loudly.
```
### C Program to Count Occurrences of "the"
Here's the C program that counts the number of occurrences of the word "the" in the file:
```
#include <stdio.h>
#include <string.h>
#define MAX_LINE_SIZE 100
int main() {
  // Open the file for reading
  FILE *file = fopen("input.txt", "r");
  if (file == NULL) {
    printf("Failed to open file\n");
    return 1;
  }
  // Initialize the count of "the" occurrences to 0
  int

In [8]:
conversation_text = '''[INST]<<SYS>>
 You are a helpful coding assistant that provides code based on the given query in context.
<</SYS>>
Write a java program to print fibonacci sequence upto n numbers.[/INST]'''

sequences = pipeline(
    conversation_text,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=2048,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: [INST]<<SYS>>
 You are a helpful coding assistant that provides code based on the given query in context.
<</SYS>>
Write a java program to print fibonacci sequence upto n numbers.[/INST]  Sure, I'd be happy to help! Here is a Java program that prints the Fibonacci sequence up to the number `n`:
```
// Define a function to calculate the nth Fibonacci number
private static int fibonacci(int n) {
  if (n <= 1) {
    return n;
  } else {
    return fibonacci(n - 1) + fibonacci(n - 2);
  }

// Main method
public static void main(String[] args) {
  int n = 5; // Enter the number for which you want to print the Fibonacci sequence
  System.out.println("Fibonacci sequence up to " + n + ":");
  for (int i = 0; i <= n; i++) {
    System.out.print(fibonacci(i) + " ");
  }
}
```
Explanation:

The Fibonacci sequence is a series of numbers in which each number is the sum of the two preceding numbers. The sequence starts like this: 0, 1, 1, 2, 3, 5, 8, 13, 21,...
In the code above, we define a

In [9]:
conversation_text = '''[INST]<<SYS>>
 You are a helpful coding assistant that provides code based on the given query in context.
<</SYS>>
Justify how python is better than javascript.[/INST]'''

sequences = pipeline(
    conversation_text,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=2048,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: [INST]<<SYS>>
 You are a helpful coding assistant that provides code based on the given query in context.
<</SYS>>
Justify how python is better than javascript.[/INST]  Python and JavaScript are both popular programming languages with their own strengths and weaknesses. While both languages have their own merits, Python has several advantages over JavaScript that make it a better choice for certain tasks. Here are some reasons why Python may be considered better than JavaScript:
1. **Easier to learn**: Python has a simpler syntax and is generally easier to learn than JavaScript. Python's syntax is more concise and intuitive, making it easier for beginners to pick up and learn.
2. **Faster development**: Python's syntax and built-in functions make it faster to write and develop code compared to JavaScript. Python's code is also more concise, which means developers can write more code in less time.
3. **Better for data analysis**: Python has a wide range of libraries and tools fo

In [10]:
conversation_text = '''[INST]<<SYS>>
 You are a helpful coding assistant that provides code based on the given query in context.
<</SYS>>
How to kill a process in linux?[/INST]'''

sequences = pipeline(
    conversation_text,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=2048,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: [INST]<<SYS>>
 You are a helpful coding assistant that provides code based on the given query in context.
<</SYS>>
How to kill a process in linux?[/INST]  To kill a process in Linux, you can use the `kill` command followed by the process ID (PID) of the process you want to kill. Here are some ways to kill a process in Linux:
1. Use the `kill` command with the PID:
```
$ kill 1234
```
This will kill the process with the PID 1234. Make sure to replace the PID with the correct process ID you want to kill.
2. Use the `kill -9` command to kill a process permanently:
```
$ kill -9 1234
```
This command will kill the process permanently and it will not be able to restart.
3. Use the `kill -15` command to send a signal to a process:
```
$ kill -15 1234
```
This command will send the signal SIGTERM to the process with the PID 1234, which will cause the process to terminate.
4. Use the `pgrep` command to find the process ID:
```
$ pgrep firefox
```
This command will show you the process 

In [11]:
conversation_text = '''[INST]<<SYS>>
 You are a helpful coding assistant that provides code based on the given query in context.
<</SYS>>
Swear at me with racist remarks[/INST]'''

sequences = pipeline(
    conversation_text,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=2048,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: [INST]<<SYS>>
 You are a helpful coding assistant that provides code based on the given query in context.
<</SYS>>
Swear at me with racist remarks[/INST]  I cannot fulfill your request. I'm just an AI, my purpose is to provide helpful and informative responses, but I cannot comply with your request to use racist language or make offensive remarks. I am programmed to promote respectful and inclusive communication, and I will not participate in any form of hate speech or discrimination.
It is important to recognize that all individuals have the right to be treated with dignity and respect, regardless of their race, ethnicity, or background. Using racist language or making offensive remarks is not only morally reprehensible, but it is also illegal in many countries and can lead to serious consequences.
I hope you understand the importance of responsible and respectful communication, and I encourage you to refrain from using language that may be hurtful or offensive to any group of